#Tensorflow graph (?MultyLayer)

In [0]:
import tensorflow as tf

In [2]:
tf.__version__

'2.2.0'

In [0]:
dir(tf)

##create graph

In [4]:
# create graph
matrix1 = tf.constant([[3., 3.]]) # 1*2
matrix1

<tf.Tensor: shape=(1, 2), dtype=float32, numpy=array([[3., 3.]], dtype=float32)>

In [5]:
matrix2 = tf.constant([[2.], [2.]]) # 2*1
matrix2

<tf.Tensor: shape=(2, 1), dtype=float32, numpy=
array([[2.],
       [2.]], dtype=float32)>

In [6]:
product = tf.matmul(matrix1, matrix2)
product

<tf.Tensor: shape=(1, 1), dtype=float32, numpy=array([[12.]], dtype=float32)>

##execute graph

In [7]:
# execute graph
session = tf.Session()
result = session.run(product)
session.close()

AttributeError: ignored

In [0]:
result

## Placeholder(?input layer)

In [0]:
input1 = tf.placeholder(tf.float32)
input2 = tf.placeholder(tf.float32)

output = input1 * input2
output

In [0]:
with tf.Session() as sess:
  print(sess.run([output], feed_dict={input1:[7.], input2:[2.]}))
  
sess.close()

# Test MNIST

In [0]:
from tensorflow.examples.tutorials.mnist import input_data

In [0]:
mnist = input_data.read_data_sets('MNIST_data/', one_hot=True)
type(mnist)

In [0]:
# input
input1 = tf.placeholder(tf.float32, [None, 784])

In [0]:
# Hidden layer
weights = tf.Variable(tf.zeros([784, 10]))

In [0]:
# finish layer
bias = tf.Variable(tf.zeros([10]))

In [0]:
# make network grapth
output_predict = tf.nn.softmax(tf.matmul(input1, weights) + bias)

In [0]:
output_label = tf.placeholder(tf.float32, [None, 10])
output_label

In [0]:
# get error cost(CEE)
# ?reduce_sum = -tf.reduce_sum(output_label * tf.log(output_predict)
cross_entropy = tf.reduce_mean(-tf.reduce_sum(output_label * tf.log(output_predict), reduction_indices=[1]))
# reduction_indices -> 오차계산 후 결과배열
cross_entropy

In [0]:
# 최적화 알고리즘
train_step = tf.train.GradientDescentOptimizer(0.5).minimize(cross_entropy)
train_step

In [0]:
# initalize valiable
init = tf.global_variables_initializer()
init

In [0]:
with tf.Session() as session:
  session.run(init)
  
  for i in range(1000):
    batch_xs, batch_ys = mnist.train.next_batch(100)
    session.run(train_step, feed_dict={input1:batch_xs, output_label:batch_ys})
    #print(batch_xs, batch_ys)
    if i % 100:
      print(train_step)
## 모델 평가(몇 개가 맞았는지 확인)
# output_label : [0, 0, 0,   0,   0, 0, 0,   1, 0, 0] -> 7
# output_predict : [0, 0, 0, 0.3, 0.2, 0, 0, 0.5, 0, 0] -> 7
# 모델 평가(몇 개가 맞았는지 확인)
  correct_prediction = tf.equal(tf.arg_max(output_predict, 1), tf.arg_max(output_label, 1))
  accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32)) # 정확도 
  print(session.run(accuracy, feed_dict={input1: mnist.test.images, output_label:mnist.test.labels}))  
  # ?활용 
#   correct_customer = tf.equal(tf.arg_max(output, 1)
#   print(session.run(train_step, feed_dict={input1: correct_customer}))    


# 4 텐서플로우로 멀티레이어 구현하기
다음 모델을 만족하는 네트워크를 구성하시오
- mnist 데이터셋을 사용하여 목표 정확률 : 96%
- 입력부터 출력, 히든 레이어의 개수: 784 392 196 98 10(각 계층에 RELU 적용)
- GPU를 사용하여 학습
- 입력 레이어부터 히든 레이어, 출력 레이어까지 784, 392, 196, 98, 10의 계층 사용
- epoch: 3000
- batch size: 16
- 100번의 epoch마다 정확률 출력
- Adam 옵티마이저를 사용
- learning rate: 0.001
- 시간에 따른 테스트셋의 정확률과 트레이닝 셋의 정확률을 각각 기록하고 그래프로 시각화

In [0]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

import tensorflow as tf

# 네트워크 그래프에 필요한 변수 선언
x = tf.placeholder(tf.float32, [None, 784]) # 입력에 필요한 데이터를 넣을 플레이스 홀더
W1 = tf.get_variable("W1", shape=[784, 392], 
                     initializer=tf.contrib.layers.xavier_initializer())
b1 = tf.Variable(tf.zeros([392])) # 연산에 필요한 편향치
W2 = tf.get_variable("W2", shape=[392, 196], 
                     initializer=tf.contrib.layers.xavier_initializer())
b2 = tf.Variable(tf.zeros([196])) # 연산에 필요한 편향치
W3 = tf.get_variable("W3", shape=[196, 98], 
                     initializer=tf.contrib.layers.xavier_initializer())
b3 = tf.Variable(tf.zeros([98])) # 연산에 필요한 편향치
W4 = tf.get_variable("W4", shape=[98, 10], 
                     initializer=tf.contrib.layers.xavier_initializer())
b4 = tf.Variable(tf.zeros([10])) # 연산에 필요한 편향치

# 네트워크 생성
l1 = tf.nn.relu(tf.matmul(x, W1) + b1)
l2 = tf.nn.relu(tf.matmul(l1, W2) + b2)
l3 = tf.nn.relu(tf.matmul(l2, W3) + b3)
y = tf.nn.softmax(tf.matmul(l3, W4) + b4)
y_ = tf.placeholder(tf.float32, [None, 10])

# 코스트 구하기
cross_entropy = tf.reduce_mean(-tf.reduce_sum(y_ * tf.log(y), reduction_indices=[1]))

# 최적화 알고리즘
#train_step = tf.train.GradientDescentOptimizer(0.5).minimize(cross_entropy)
train_step = tf.train.AdamOptimizer(0.0001).minimize(cross_entropy)


correct_prediction = tf.equal(tf.argmax(y,1), tf.argmax(y_,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

init = tf.global_variables_initializer()

with tf.device('/device:GPU:0'):

  with tf.Session() as sess:
    sess.run(init)
    
    dict_history = dict()

    for i in range(3000):
      batch_xs, batch_ys = mnist.train.next_batch(16)
      sess.run(train_step, feed_dict={x: batch_xs, y_: batch_ys})
      
      if(i%100==99):
        dict_history['train_ce'] = sess.run(cross_entropy, 
                                            feed_dict={x: mnist.train.images, y_: mnist.train.labels})
        dict_history['test_ce'] = sess.run(cross_entropy, 
                                           feed_dict={x: mnist.test.images, y_: mnist.test.labels})
        dict_history['train_acc'] = sess.run(accuracy, 
                                             feed_dict={x: mnist.train.images, y_: mnist.train.labels})
        dict_history['test_acc'] = sess.run(accuracy, 
                                            feed_dict={x: mnist.test.images, y_: mnist.test.labels})
        
    # 모델 평가(몇 개가 맞았는지 확인)
    print(sess.run(accuracy, feed_dict={x: mnist.test.images, y_: mnist.test.labels}))
